In [3]:
import network
import mnist_loader

In [8]:
training_data, validation_data, test_data = mnist_loader.load_data()

In [9]:
type(training_data)

tuple

In [11]:
training_data[1]

array([5, 0, 4, ..., 8, 4, 8])

In [16]:
print len(training_data[0])
print len(training_data[1])
print len(training_data)

50000
50000
2


In [14]:
from collections import defaultdict
T = defaultdict(list)

In [18]:
for img,digit in zip(training_data[0],training_data[1]):
    T[digit].append(img)

In [21]:
# making sure our d is what we expect it to be
print T.keys()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [22]:
# Checking the cardinality of T_d for each d
#for digit in T.keys():
for digit in range(0,10):
    print len(T[digit])

4932
5678
4968
5101
4859
4506
4951
5175
4842
4988
